#$\text{Error control for reduced order models (a posteriori error estimator)}$

✅ Fase 1 – Preparazione
Obiettivo: Consolidare le basi teoriche e tecniche.

Studio approfondito di CRBM Capitoli 3-4

Cap. 3: Reduced Basis Methods – concetti di manifold, offline-online decomposition, POD, greedy algorithm.

Cap. 4: Certified Error Control – stime a posteriori, costanti di coercività, effetto dell’errore sulla riduzione.

Definizione del problema fisico

Scegli un problema ellittico parametrizzato (es. Poisson, conduzione, elasticità).

Introduci condizioni al bordo di Dirichlet non omogenee, eventualmente con una lifting function affine nel parametro.

Assicurati che sia ammissibile una decomposizione affina.

✅ Fase 2 – Implementazione base (truth model)
Obiettivo: Costruire il “truth” solver ad alta fedeltà.

Costruzione di un solver FEM

Scelta tra:

FEM 1D/2D scritto da te (es. con NumPy/SciPy).

FEM 2D con FEniCS (fortemente consigliato per facilità).

Parametrizza uno o più aspetti del problema (coefficiente diffusivo, condizioni al bordo, forma del dominio...).

✅ Fase 3 – Costruzione del modello ridotto
Obiettivo: Generare una base ridotta tramite POD o Greedy.

Offline stage

Costruzione snapshot: risolvi il problema per un insieme di parametri.

Costruisci la base ridotta:

POD: tramite SVD degli snapshot.

Greedy: usa un estimatore dell’errore a posteriori (da Cap. 4).

Ortonormalizzazione delle basi (es. Gram-Schmidt).

Precomputazione della decomposizione affine (matrici ridotte Aq_rb, vettori fq_rb...).

Online stage

Per ogni nuovo parametro:

Assemblaggio rapido della matrice ridotta.

Soluzione del sistema ridotto.

Calcolo dell’output e dell’error estimator.

✅ Fase 4 – Analisi dell’errore e certificazione
Obiettivo: Validare il modello ridotto e l'estimatore.

Plot e analisi dei risultati

Errore vs. numero di basi ridotte.

Error estimator vs. numero di basi.

Fissato N: errore e stimatore al variare del parametro.

Effetto della costante di coercività (confronta stime conservative vs. reali).

Indice di effectivity dell’estimatore.

✅ Fase 5 – Analisi computazionale
Obiettivo: Dimostrare i vantaggi del modello ridotto.

Benchmark di tempo

Tempo medio di una simulazione “truth” vs. ridotta.

Calcolo dello speedup.

Mostra la scalabilità con il numero di parametri.

✅ Fase 6 – Conclusione e report
Obiettivo: Comunicare risultati e riflessioni.

Documentazione finale

Descrizione del problema, solutore, implementazione ridotta.

Dettagli sugli estimator a posteriori.

Grafici e interpretazione dei risultati.

Limiti osservati e possibili estensioni future.

In [ ]:
#@title import dependencies

# Installing FEniCS (dolfin) on the Google Colab servers
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin
from dolfin import *

# Importing some libraries
from dolfin import * # This is the core of FEniCS and it contains all the FEM functions we will need
from ufl_legacy.geometry import * # This helps in designing geometries
from dolfin.cpp.mesh import *     # This handles meshes
from mshr import *                # This generates meshes

## Definizione del problema fisico

Definiamo un problema fisico concreto su cui applicare la riduzione di ordine con controllo dell’errore.

Consideriamo l'equazione di Poisson 2D. la **forma forte** è descritta dal sistema

$$
\begin{cases}
- \nabla \cdot (a(x;\mu)\nabla u(x;\mu)) = f(x) & \text{in } \Omega = (0,1)^2 \\
u(x;\mu) = g(x;\mu) & \text{su } \partial \Omega
\end{cases}
$$

e, dato $\mu = (\mu_1, \mu_2) \in \mathcal{P} = [0.1, 1] \times [0, 1]$, scegliamo:

- Diffusività:$\quad a(x; \mu) = 1 + \mu_1 x_1 $ \
(affine rispetto a $\mu \ \Rightarrow$ utile per la riduzione)

- Dirichlet's non-homogeneus BC:$\quad g(x;\mu) = \mu_2 \cdot \sin(\pi x_1) \sin(\pi x_2) $ \
 (affine rispetto a $\mu \ \Rightarrow$ lifting facile)

- Forzante (termine sorgente):$\quad f(x) = 1 $

\
Per gestire la non omogeneità delle condizioni al bordo utilizziamo una funzione **"Lifting"** per ricondurci al problema omogeneo, quindi:
$$
u_{\text{lift}} := g(x;\mu) \text{ su } \partial\Omega
$$
ora, definendo $ \tilde{u}(x;\mu) = u(x;\mu) - u_{\text{lift}}(x;\mu) $ otteniamo che $ \tilde{u} $ risolve il seguente sistema **omogeneo**:

$$
\begin{cases}
- \nabla \cdot (a(x;\mu)\nabla \tilde{u}(x;\mu)) = - \nabla \cdot (a(x;\mu)\nabla u(x;\mu)) = f(x) & \text{in } \Omega = (0,1)^2 \\
\tilde{u}(x;\mu) = u(x;\mu) - u_{\text{lift}}(x;\mu) = g(x;\mu) - g(x;\mu) = 0 & \text{su } \partial \Omega
\end{cases}
$$

\
A questo punto possiamo passare alla **forma debole**, cioè:
Trova $\tilde{u}(\mu) \in H_0^1(\Omega)$ tale che:

$$
\int_\Omega a(x;\mu) \nabla \tilde{u} \cdot \nabla v \ dx + \int_\Omega a(x;\mu) \nabla u_{\text{lift}} \cdot \nabla v \ dx = \int_\Omega f v \ dx,  \quad \forall v \in H_0^1(\Omega)
$$
(ricordando che $ u = \tilde{u} + u_{lift} $)


In [ ]:
#@title Geometria

Omega = Rectangle(Point(0, 0), Point(1, 1))
# Define a mesh with N = 32 points on an edge
mesh = generate_mesh(domain, 32)

Proprietà utile:
Il problema è **coercivo** (Poisson su dominio limitato con $a(x;\mu) \geq 0.1$) \
- Ricordiamo che:\
data una forma bilineare $a(u,v;\mu)$ diciamo che questa è coerciva se $a(u,v;\mu) \geq \alpha(\mu) \ ||v||^2_{H_0^1}$ con $\alpha(\mu)$coefficiente di coercività.
- in altre parole:
  - $a(\cdot,\cdot;\mu)$ controlla la norma $ ||v||$
  - Nessuna "direzione" di $v$ può sfuggire al controllo (nessuna degenerazione)

\
Nel nostro caso abbiamo:

1. $a(x;\mu) \geq \underline{a} > 0$ per ogni $x \in \Omega$, $\mu \in \mathcal{P}$  
   - Ad esempio: $a(x;\mu) = 1 + \mu_1 x_1 \geq 1$ (dato che $\mu_1 \geq 0.1$, $x_1 \in [0,1]$)  
   - $\Rightarrow a(x;\mu) \geq 1 \Rightarrow \alpha(\mu) \geq 1$

2. Il dominio $\Omega$ è limitato e regolare, quindi:
   - Poincaré: $\|v\|_{L^2} \leq C \|\nabla v\|$
   - $\Rightarrow$ la norma $H^1_0$ è equivalente a $\|\nabla v\|$

3. La forma bilineare è:
   $$
   a(u,v;\mu) = \int_\Omega a(x;\mu) \nabla u \cdot \nabla v \, dx
   $$
   che è **simmetrica**, **positiva definita**, e **uniformemente ellittica**

\
Vantaggi:
- Esistenza e unicità della soluzione per il teorema di Lax-Milgram
- Stabilità numerica per Finite Element Method (FEM)
- Errori stimabili a posteriori
- RBM certificabile cioè è possibile stimare l’errore tra soluzione ridotta e completa
- Convergenza garantita: Greedy converge rapidamente se l'errore è controllato da coercività

\
Senza coercività?

Se $\alpha(\mu) \to 0$ (cioè $a(x;\mu)$ vicino a 0):

- Il problema diventa quasi-degenerato
- L’errore esplode (stimatore → infinito)
- FEM diventa instabile
- RBM non può certificare nulla
---

\

Obiettivo nel contesto RBM

- Costruire base ridotta (POD o greedy)
- Stimare errore e certificare $\eta(\mu)$
- Visualizzare:
  - errore vs. $N$ basi
  - errore/estimatore vs. parametro
  - speedup vs. full FEM